This script use kirita data to annotate the cellbender-corrected count

1. PopV integration

- Initial Date: 12/8/2025

In [1]:
# Set 'Liberation Sans' as the default font
from matplotlib import rcParams
rcParams['font.family'] = 'Liberation Sans'

In [2]:
#!/usr/bin/env python
"""
Standard Scanpy Analysis Pipeline for D0_1 and D3_1 samples
Following: https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html

No integration step - just standard preprocessing, QC, normalization, 
dimensionality reduction, and clustering.
"""

import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os

# Set scanpy settings
sc.settings.set_figure_params(dpi=200, facecolor='white', figsize=(6, 5))
sc.settings.verbosity = 3

# =============================================================================
# CONFIGURATION - Update these paths
# =============================================================================
# Input: Combined h5ad file with D0_1 and D3_1 samples
INPUT_H5AD = "/home/data3/dianli/projects/kenji_rhabdo_2025/processed_data/cellbender_count.h5ad"

ref_H5AD = "/home/data3/dianli/projects/kenji_rhabdo_2025/processed_data/kirita_126578_cell_raw_count.h5ad"
# Output directory
MODEL_OUTPUT_DIR = "/home/data3/dianli/projects/kenji_rhabdo_2025/processed_data/analysis_models/cellbender_count/label_transfer/popv/"
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# Sample column name
SAMPLE_COL = "sample"  # or "sample" depending on your data

# Samples to subset
SAMPLES_TO_USE = ["D0_1", "D3_1"]

# =============================================================================
# 1. LOAD DATA AND SUBSET TO D0_1 AND D3_1
# =============================================================================
print("=" * 60)
print("1. LOADING AND SUBSETTING DATA")
print("=" * 60)

# Load the query adata
adata_query = sc.read_h5ad(INPUT_H5AD)
print(f"Loaded adata: {adata_query.shape[0]} cells x {adata_query.shape[1]} genes")

# Load the ref adata
adata_ref = sc.read_h5ad(ref_H5AD)
print(f"Loaded adata: {adata_ref.shape[0]} cells x {adata_ref.shape[1]} genes")

# 1. Ensure your data is loaded (example placeholders)
# adata_query = sc.read("query_data.h5ad")
# adata_ref = sc.read("reference_data.h5ad")

1. LOADING AND SUBSETTING DATA
Loaded adata: 22612 cells x 32285 genes
Loaded adata: 126578 cells x 27133 genes


In [3]:
import scanpy as sc
import popv
from popv.preprocessing import Process_Query
from popv.annotation import annotate_data
from datetime import datetime

# =============================================================================
# 2. Map your cell type labels to Cell Ontology terms
# =============================================================================
# Customize this mapping based on your reference labels
# label_mapping = {
#     # Proximal tubule segments
#     "PT": "epithelial cell of proximal tubule",
#     "PT-S1": "epithelial cell of proximal tubule",
#     "PT-S2": "epithelial cell of proximal tubule",
#     "PT-S3": "epithelial cell of proximal tubule",
#     "PTS1": "epithelial cell of proximal tubule",
#     "PTS2": "epithelial cell of proximal tubule",
#     "PTS3": "epithelial cell of proximal tubule",
#     "PCT": "epithelial cell of proximal tubule",
#     "PST": "epithelial cell of proximal tubule",
    
#     # Loop of Henle
#     "DTL": "kidney loop of Henle thin descending limb epithelial cell",
#     "ATL": "kidney loop of Henle thin ascending limb epithelial cell",
#     "TAL": "kidney loop of Henle thick ascending limb epithelial cell",
#     "tAL": "kidney loop of Henle thin ascending limb epithelial cell",
#     "LOH": "kidney loop of Henle epithelial cell",
    
#     # Distal nephron
#     "DCT": "kidney distal convoluted tubule epithelial cell",
#     "CNT": "kidney connecting tubule epithelial cell",
#     "PC": "renal principal cell",
#     "IC": "renal intercalated cell",
#     "IC-A": "renal intercalated cell",
#     "IC-B": "renal intercalated cell",
#     "CD": "kidney collecting duct epithelial cell",
    
#     # Glomerular
#     "POD": "glomerular visceral epithelial cell",
#     "Podocyte": "glomerular visceral epithelial cell",
#     "PEC": "parietal epithelial cell",
#     "EC-GC": "glomerular endothelial cell",
#     "Mesangial": "mesangial cell",
    
#     # Endothelial subtypes
#     "EC": "endothelial cell",
#     "EC-AEA": "endothelial cell",
#     "EC-DVR": "endothelial cell",
#     "EC-AVR": "endothelial cell",
#     "EC-PTC": "endothelial cell",
#     "EC-LYM": "lymphatic endothelial cell",
    
#     # Stromal
#     "FIB": "fibroblast",
#     "Fibroblast": "fibroblast",
#     "Myofibroblast": "myofibroblast",
#     "vSMC": "vascular associated smooth muscle cell",
#     "Pericyte": "pericyte",
    
#     # Immune
#     "Macrophage": "macrophage",
#     "MAC": "macrophage",
#     "Monocyte": "monocyte",
#     "T cell": "T cell",
#     "NK": "natural killer cell",
#     "B cell": "B cell",
#     "Plasma": "plasma cell",
#     "DC": "dendritic cell",
#     "Mast": "mast cell",
#     "Neutrophil": "neutrophil",
# }

# Apply mapping to reference
# adata_ref.obs["cell_ontology_class"] = adata_ref.obs["celltype"].map(label_mapping)

# Check for unmapped labels
# unmapped = adata_ref.obs[adata_ref.obs["cell_ontology_class"].isna()]["celltype"].unique()
# if len(unmapped) > 0:
#     print(f"WARNING: Unmapped labels found: {unmapped}")
#     print("Please add these to label_mapping dictionary above!")
# else:
#     print("All labels successfully mapped to Cell Ontology terms.")

# =============================================================================
# 3. Configure popV settings (optional)
# =============================================================================
popv.settings.compute_embedding = True    # Compute UMAP embeddings
popv.settings.return_probabilities = True # Return certainty scores

# =============================================================================
# 4. Preprocess query and reference data
# =============================================================================
adata = Process_Query(
    query_adata=adata_query,
    ref_adata=adata_ref,
    ref_labels_key="celltype",              # Your cell type column directly
    ref_batch_key="orig.ident",             # Key for batch info in ref (or None)
    query_batch_key=None,                   # Query batch key if applicable
    cl_obo_folder=False,                    # Disable Cell Ontology and OnClass
    prediction_mode="retrain",              # Options: "retrain", "inference", "fast"
    n_samples_per_label=300,                # Subsample reference per cell type
    save_path_trained_models=MODEL_OUTPUT_DIR,
    hvg=4000,
).adata
# adata = Process_Query(
#     adata_query=adata_query,
#     adata_ref=adata_ref,
#     ref_labels_key="cell_ontology_class",  # Use the mapped column
#     ref_batch_key="orig.ident",                  # Key for batch info in ref (or None)
#     query_batch_key=None,                   # Query batch key if applicable
#     cl_obo_folder=True,                     # Use Cell Ontology for OnClass
#     prediction_mode="retrain",              # Options: "retrain", "inference", "fast"
#     n_samples_per_label=300,                # Subsample reference per cell type
#     save_path_trained_models=MODEL_OUTPUT_DIR,     # Where to save trained models
#     hvg=4000,                               # Number of highly variable genes
# ).adata

# =============================================================================
# 5. Run annotation
# =============================================================================
print(datetime.now())

annotate_data(adata)

print(datetime.now())



Loading faiss with AVX512 support.
Successfully loaded faiss with AVX512 support.
2025-12-08 18:32:21.660573: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-08 18:32:21.723700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-08 18:32:22.845565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`

filtered out 9695 genes that are detected in less than 200 cells
extracting highly variable genes
normalizing counts per cell
    finished (0:00:00)


Sampling 300 cells per label


... be careful when using `max_value` without `zero_center`.
computing PCA
    with n_comps=50
    finished (0:00:05)
filtered out 414 cells that have less than 30 counts


The following cells will be excluded from annotation because they have low expression:Index(['TCACGGATCGCATCCT-1_1', 'TCAAGAACAACTGGCT-1_1', 'AAGCCTGTCGTTAACA-1_1',
       'GTAGGAGGTTGTCCAT-1_1', 'TCCTTAGTCTAAATCG-1_1', 'TGTGGCCAGAACCTAC-1_1',
       'CTTGCAACAGACAAAC-1_1', 'CGTAACTAGCTGTCAG-1_1', 'ACTAACTCATTAGCCA-1_1',
       'GCAGGCTTCTCGCCTG-1_1',
       ...
       'TCGGTTCTCATGAAGG-1_2', 'CAGCATTAGGGTCTAT-1_2', 'TTCATCCCAAACTAAG-1_2',
       'GAGAACCAGCGTGCAC-1_2', 'AGTTGCAGTCCTGATT-1_2', 'ATGTCATCAGGCCTTG-1_2',
       'AGCCTGGGTACCGTTT-1_2', 'GTTCCCAGTAGCCTCC-1_2', 'CCTGCTCCAGGTTTGC-1_2',
       'GGAACCTGTGTGTCCC-1_2'],
      dtype='object', length=414).


...

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


Saving UMAP of BBKNN results to adata.obsm["X_umap_scanvi_popv"]


computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:25)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP parameters (adata.uns) (0:03:05)


100%|███████████████████████████████████████████████████████████████████████████████████████████| 7/7 [46:16<00:00, 396.58s/it]
Using predictions ['popv_celltypist_prediction', 'popv_knn_bbknn_prediction', 'popv_knn_harmony_prediction', 'popv_knn_on_scvi_prediction', 'popv_scanvi_prediction', 'popv_svm_prediction', 'popv_xgboost_prediction'] for PopV consensus


2025-12-08 19:19:10.192823


In [ ]:
# =============================================================================
# 6. Access results
# =============================================================================
# Results are stored in adata.obs with these columns:
# - popv_majority_vote_prediction: consensus cell type
# - popv_majority_vote_score: number of algorithms that agreed (confidence)
# - Individual predictions: popv_svm_prediction, popv_rf_prediction, etc.

# Get query cells only
query_results = adata[adata.obs["_query"]].copy()

# Check predictions
print("\n=== Cell Type Predictions ===")
print(query_results.obs["popv_majority_vote_prediction"].value_counts())

print("\n=== Confidence Scores (higher = more algorithms agreed) ===")
print(query_results.obs["popv_majority_vote_score"].value_counts().sort_index())

In [ ]:
# =============================================================================
# 7. Save results
# =============================================================================
# Save full annotated object
# adata.write_h5ad("popv_annotated_full.h5ad")

# Save query results only
# query_results.write_h5ad("popv_annotated_query.h5ad")

# Export predictions to CSV
# query_results.obs[[
#     "popv_majority_vote_prediction",
#     "popv_majority_vote_score"
# ]].to_csv("popv_predictions.csv")

# =============================================================================
# 8. Visualize results (optional)
# =============================================================================
# import matplotlib.pyplot as plt

# # UMAP colored by predictions
# sc.pl.umap(query_results, color=["popv_majority_vote_prediction"], 
#            title="PopV Cell Type Predictions")
# sc.pl.umap(query_results, color=["popv_majority_vote_score"], 
#            title="PopV Confidence Score", cmap="viridis")

# # Confidence distribution
# fig, ax = plt.subplots(figsize=(8, 5))
# query_results.obs["popv_majority_vote_score"].value_counts().sort_index().plot(kind='bar', ax=ax)
# ax.set_xlabel("Number of methods agreeing")
# ax.set_ylabel("Number of cells")
# ax.set_title("PopV Confidence Distribution")
# plt.tight_layout()
# plt.savefig("popv_confidence_distribution.png", dpi=150)
# plt.show()

# =============================================================================
# 9. Filter by confidence (optional)
# =============================================================================
# # High confidence: 6+ methods agree
# high_conf = query_results[query_results.obs["popv_majority_vote_score"] >= 6].copy()
# print(f"\nHigh confidence cells: {high_conf.n_obs}")

# # Low confidence: fewer than 4 methods agree (may need manual review)
# low_conf = query_results[query_results.obs["popv_majority_vote_score"] < 4].copy()
# print(f"Low confidence cells (review these): {low_conf.n_obs}")